## 5310 Matches

In [1]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [2]:
pd.set_option("display.max_columns", 100)

In [40]:
df1 = pd.read_csv('5310_org_rd1match.csv')

In [4]:
df2 = pd.read_excel('5310_unmatched.xls')

In [5]:
df = pd.concat([df1, df2], ignore_index=True, sort=False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   organization_name           150 non-null    object 
 1   ntd_id_x                    6 non-null      object 
 2   ntd_id_y                    6 non-null      object 
 3   ntd_id                      29 non-null     object 
 4   agency_name                 35 non-null     object 
 5   name                        109 non-null    object 
 6   doing_business_as           7 non-null      object 
 7   reporter_acronym            27 non-null     object 
 8   alias                       0 non-null      float64
 9   organization_type_x         42 non-null     object 
 10  organization_type_y         86 non-null     object 
 11  roles                       5 non-null      object 
 12  reporter_type               35 non-null     object 
 13  city                        35 non-

In [7]:
df.reported_by_ntd_id.info()

<class 'pandas.core.series.Series'>
RangeIndex: 150 entries, 0 to 149
Series name: reported_by_ntd_id
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 1.3 KB


In [8]:
df.sample()

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,alias,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,compare_acronym,compare_names,in_5310,reported_by_ntd_id,Unnamed: 11
28,LIFE ElderCare,NaN,NaN,NaN,NaN,LIFE ElderCare,NaN,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,437.0,NaN,Life ElderCare,NaN,right_only,NaN,NaN,both,NaN,NaN


In [9]:
#getting rid of the compare cols so we can check again. and some other columns
df = df.drop(columns={'compare_acronym','compare_names','reported_by_ntd_id','Unnamed: 11','alias'})

In [10]:
compare_col2 = np.where(df["organization_name"] == df["name"], True, False)
df["compare_names"] = compare_col2
df.compare_names.value_counts()

False    84
True     66
Name: compare_names, dtype: int64

In [11]:
df.ntd_id.fillna(df['ntd_id_x'], inplace=True)
df.ntd_id.fillna(df['ntd_id_y'], inplace=True)

In [12]:
df.sample()

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names
144,Golden Rain Foundation of Walnut Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,False


In [13]:
df._merge.value_counts()

right_only    74
both          35
Name: _merge, dtype: int64

In [14]:
df.in_5310.value_counts()

left_only    84
both         66
Name: in_5310, dtype: int64

In [15]:
def recategorize(df):   
    if (df['_merge']=='right_only') and (df['in_5310'] == 'both'):
        return 'Name Match; Airtable'
    elif (df['_merge']=='both') and (df['in_5310'] == 'both'):
        return 'Name Match; NTD'
    elif pd.isnull(df['_merge']) and (df['in_5310'] == 'left_only'):
        return 'No Match'
    elif (df['_merge']=='right_only') and (df['in_5310'] == 'left_only'):
        return 'Manual Match, Airtable'
    elif (df['_merge']=='both') and (df['in_5310'] == 'left_only'):
        return 'Manual Match, NTD'
    else: 
        return "NaN"

In [16]:
df['merge_status'] = df.apply(recategorize, axis = 1)

In [17]:
df

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
0,Angel View Inc,NaN,NaN,NaN,NaN,Angel View Inc,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,NaN,NaN,Angel View,NaN,right_only,both,True,Name Match; Airtable
1,ARC Imperial Valley,NaN,NaN,NaN,NaN,ARC Imperial Valley,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,398.0,NaN,Arc Imperial Valley Transportation,NaN,right_only,both,True,Name Match; Airtable
2,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
3,Big Valley 50 Plus,NaN,NaN,NaN,NaN,Big Valley 50 Plus,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,404.0,NaN,Big Valley 50 Plus,NaN,right_only,both,True,Name Match; Airtable
4,Care-A-Van Transit Systems Inc,NaN,NaN,NaN,NaN,Care-A-Van Transit Systems Inc,NaN,NaN,NaN,Company,NaN,NaN,NaN,407.0,NaN,Care-A-Van Transit,NaN,right_only,both,True,Name Match; Airtable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Loma Linda University Health,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,False,No Match
146,New Advances for People with Disabilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,False,No Match
147,Pomeroy Recreation & Rehabilitation Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,False,No Match
148,Regents of the University of California Santa ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,False,No Match


In [18]:
df.merge_status.value_counts()

No Match                  41
Name Match; Airtable      37
Manual Match, Airtable    37
Name Match; NTD           29
Manual Match, NTD          6
Name: merge_status, dtype: int64

In [19]:
df.compare_names.value_counts()

False    84
True     66
Name: compare_names, dtype: int64

In [20]:
#df = df.drop(columns={'agency_name','ntd_id_x','ntd_id_y', '_merge', 'in_5310'})

In [21]:
(df>>select(_.organization_type_x, _.organization_type_y)).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   organization_type_x  42 non-null     object
 1   organization_type_y  86 non-null     object
dtypes: object(2)
memory usage: 2.5+ KB


In [22]:
df.organization_type_y.fillna(df['organization_type_x'], inplace=True)


In [23]:
df>>filter(_.organization_type_x.notnull())

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
8,City of Rio Vista,NaN,NaN,9R02-91014,City of Rio Vista,City of Rio Vista,Rio Vista Delta Breeze,NaN,"City, County or Local Government Unit or Depar...",City/Town,NaN,Rural Reporter,Rio Vista,264.0,448003.0,Rio Vista Delta Breeze,Metropolitan Transportation Commission,both,both,True,Name Match; NTD
9,City of Roseville,NaN,NaN,90168,City of Roseville,City of Roseville,Roseville Transit,NaN,"City, County or Local Government Unit or Depar...",City/Town,NaN,Reduced Reporter,Roseville,271.0,331005.0,"Roseville Transit,Roseville Dial-A-Ride",Sacramento Area Council of Governments,both,both,True,Name Match; NTD
10,City of Santa Rosa,NaN,NaN,90017,City of Santa Rosa,City of Santa Rosa,NaN,NaN,"City, County or Local Government Unit or Depar...",City/Town,NaN,Full Reporter,Santa Rosa,301.0,449002.0,"Santa Rosa CityBus,Santa Rosa Call Ahead Service",Metropolitan Transportation Commission,both,both,True,Name Match; NTD
11,City of Visalia,NaN,NaN,90091,City of Visalia,City of Visalia,Visalia Transit,VT,"City, County or Local Government Unit or Depar...",City/Town,NaN,Full Reporter,Visalia,361.0,654006.0,"V-LINE,Visalia Transit,Visalia Dial-A-Ride",Tulare County Association of Governments,both,both,True,Name Match; NTD
13,Colusa County Transit Agency,NaN,NaN,9R02-91112,Colusa County Transit Agency,Colusa County Transit Agency,NaN,CCTA,"City, County or Local Government Unit or Depar...",Independent Agency,NaN,Rural Reporter,Colusa,74.0,306002.0,Colusa County Transit,NaN,both,both,True,Name Match; NTD
17,Eastern Sierra Transit Authority,NaN,NaN,9R02-91062,Eastern Sierra Transit Authority,Eastern Sierra Transit Authority,NaN,ESTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Rural Reporter,Bishop,99.0,926002.0,Eastern Sierra Transit Authority Community Rou...,NaN,both,both,True,Name Match; NTD
18,El Dorado County Transit Authority,NaN,NaN,90229,El Dorado County Transit Authority,El Dorado County Transit Authority,NaN,EDCTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Reduced Reporter,Diamond Springs,101.0,309002.0,"El Dorado Transit,Sacramento/South Lake Tahoe ...",Sacramento Area Council of Governments,both,both,True,Name Match; NTD
22,Humboldt Transit Authority,NaN,NaN,9R02-91036,Humboldt Transit Authority,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Rural Reporter,Eureka,135.0,112005.0,"Humboldt Transit Authority Dial-A-Ride,Redwood...",NaN,both,both,True,Name Match; NTD
23,Humboldt Transit Authority,NaN,NaN,A0009,Humboldt Transit Authority,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Group Plan Sponsor,Eureka,135.0,112005.0,"Humboldt Transit Authority Dial-A-Ride,Redwood...",NaN,both,both,True,Name Match; NTD
24,Imperial County Transportation Commission,NaN,NaN,90226,Imperial County Transportation Commission,Imperial County Transportation Commission,NaN,ICTC,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Full Reporter,El Centro,138.0,1113001.0,Imperial Valley Transit,Southern California Association of Governments,both,both,True,Name Match; NTD


In [24]:
df>>filter(_.organization_type_y.isnull())

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
2,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
15,Desert Access and Mobility Inc,NaN,NaN,NaN,NaN,Desert Access and Mobility Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
25,Independent Living Partnership,NaN,NaN,NaN,NaN,Independent Living Partnership,NaN,NaN,NaN,NaN,NaN,NaN,NaN,433.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
38,North and South of Market Adult Day Health,NaN,NaN,NaN,NaN,North and South of Market Adult Day Health,NaN,NaN,NaN,NaN,NaN,NaN,NaN,445.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
39,North Valley Services,NaN,NaN,NaN,NaN,North Valley Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
62,Vivalon Inc,NaN,NaN,NaN,NaN,Vivalon Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golden Gate Transit Paratransit,NaN,right_only,both,True,Name Match; Airtable
73,Avenidas,NaN,NaN,NaN,NaN,Avenidas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401.0,NaN,Avenidas Door-to-Door,NaN,right_only,left_only,False,"Manual Match, Airtable"
77,Community Bridges Liftline,NaN,NaN,NaN,NaN,Community Bridges Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420.0,NaN,Lift Line,NaN,right_only,left_only,False,"Manual Match, Airtable"
78,Community Partners In Caring,NaN,NaN,NaN,NaN,Partners in Caring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only,left_only,False,"Manual Match, Airtable"
79,Dignity Health Connected Living,NaN,NaN,NaN,NaN,Dignity Health,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.0,NaN,"Dignity Health Connected Living,Shasta Connect",NaN,right_only,left_only,False,"Manual Match, Airtable"


## Organization_types 

In [25]:
print(f"Of the {len(df)} Organizations using 5310 funds, {len(df>>filter(_.organization_type_y.isnull()))} have no org type information")

Of the 150 Organizations using 5310 funds, 57 have no org type information


In [26]:
df.organization_type_y.value_counts()

Non-Profit Organization                        48
Independent Agency                             27
City/Town                                      10
Company                                         2
MPO/RTPA                                        2
Council of Governments                          1
Joint Powers Agency                             1
Private-Non-Profit Corporation                  1
Consolidated Transportation Services Agency     1
Name: organization_type_y, dtype: int64

#### Non-Profit

In [27]:
print(len(df>>filter(_.organization_type_y=='Non-Profit Organization')>>select(_.organization_name)))

48


In [28]:
df>>filter(_.organization_type_y=='Non-Profit Organization')>>select(_.organization_name)

,organization_name
0,Angel View Inc
1,ARC Imperial Valley
3,Big Valley 50 Plus
5,Casa Allegra Community Services
6,Catholic Charities of the Diocese of Stockton
14,Common Ground Senior Services
16,Desert ARC
19,ElderHelp of San Diego
20,Friends of Children with Special Needs
21,Humboldt Senior Resource Center


#### Transit Provider (with NTDs)

In [29]:
print(len(df>>filter(_.ntd_id.notnull())>>select(_.organization_name)))

36


In [30]:
df>>filter(_.ntd_id.notnull())>>select(_.organization_name)

,organization_name
8,City of Rio Vista
9,City of Roseville
10,City of Santa Rosa
11,City of Visalia
12,City of Visalia
13,Colusa County Transit Agency
17,Eastern Sierra Transit Authority
18,El Dorado County Transit Authority
22,Humboldt Transit Authority
23,Humboldt Transit Authority


#### Independent Agency

In [31]:
len(df>>filter(_.organization_type_y=='Independent Agency')>>select(_.organization_name))

27

In [32]:
df>>filter(_.organization_type_y=='Independent Agency')>>select(_.organization_name)

,organization_name
13,Colusa County Transit Agency
17,Eastern Sierra Transit Authority
18,El Dorado County Transit Authority
22,Humboldt Transit Authority
23,Humboldt Transit Authority
24,Imperial County Transportation Commission
27,Lake Transit Authority
29,Marin County Transit District
30,Mendocino Transit Authority
34,Monterey-Salinas Transit


#### City or Town

In [33]:
len(df>>filter(_.organization_type_y=='City/Town')>>select(_.organization_name))

10

In [34]:
df>>filter(_.organization_type_y=='City/Town')>>select(_.organization_name)

,organization_name
7,City of Fontana
8,City of Rio Vista
9,City of Roseville
10,City of Santa Rosa
11,City of Visalia
12,City of Visalia
58,Town of Truckee
71,San Francisco Municipal Transportation Agency
74,City of Lafayette: Lamorinda Sprit Van Program
75,City of Vista


### Still unknown

In [36]:
df>>filter(_.merge_status=='No Match')>>select(_.organization_name)

,organization_name
109,Choice in Aging
110,Consolidated Tribal Health Project Inc
111,County of Sacramento Department of Child Famil...
112,County of Sonoma Human Services Department Adu...
113,County of Ventura Health Services Agency
114,Dignity Community Care
115,Foothill AIDS Project
116,Mental Health Client Action Network (MHCAN)
117,NCI Affiliates
118,OPARC


In [39]:
#df.to_csv('5310_org_info.csv')